In [18]:
import time
from datetime import datetime, timedelta
import subprocess
import re
import datetime
from datetime import timedelta
import whisper
import imageio
from tqdm import tqdm
import os

In [2]:
from xinference.client import Client
client = Client("http://192.168.1.207:9997")
model = client.get_model("whisper-medium")
#with open("/public/home/liujunwu/workdir/Voice/1.wav", "rb") as audio_file:
#   model.transcriptions(audio=audio_file.read())

/public/home/liujunwu/software/miniconda3/envs/xinference/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-15 14:41:29,254	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [24]:
model = whisper.load_model('small',download_root='/public/home/liujunwu/workdir/Voice')

In [4]:
from moviepy.editor import AudioFileClip

file_mp4 = '/public/home/liujunwu/workdir/Voice/benching.mp4'
file_wav = '/public/home/liujunwu/workdir/Voice/1benching.wav'
# 先提取出音轨wav

my_audio = AudioFileClip(file_url)
my_audio.write_audiofile(file_wav)

ModuleNotFoundError: No module named 'moviepy'

In [3]:
# 将毫秒转为字幕格式时间值
def ms_to_time_string(*, ms=0, seconds=None):
    # 计算小时、分钟、秒和毫秒
    if seconds is None:
        td = timedelta(milliseconds=ms)
    else:
        td = timedelta(seconds=seconds)
    hours, remainder = divmod(td.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = td.microseconds // 1000

    time_string = f"{str(hours).zfill(2)}:{str(minutes).zfill(2)}:{str(seconds).zfill(2)},{str(milliseconds).zfill(3)}"
    return time_string

In [13]:
def seconds_to_hmsm(seconds):
    """
    输入一个秒数，输出为H:M:S:M时间格式
    @params:
        seconds   - Required  : 秒 (float)
    """
    hours = str(int(seconds // 3600))
    minutes = str(int((seconds % 3600) // 60))
    seconds = seconds % 60
    milliseconds = str(int(int((seconds - int(seconds)) * 1000))) # 毫秒留三位
    seconds = str(int(seconds))
    # 补0
    if len(hours) < 2:
        hours = '0' + hours
    if len(minutes) < 2:
        minutes = '0' + minutes
    if len(seconds) < 2:
        seconds = '0' + seconds
    if len(milliseconds) < 3:
        milliseconds = '0'*(3-len(milliseconds)) + milliseconds
    return f"{hours}:{minutes}:{seconds},{milliseconds}"

In [46]:
mp4_files = ['Sample Management Recording.mp4','Data CurationOffice Hours Recording.mp4']
for file in tqdm(mp4_files):
    # 字幕文件保存路径
    # xxx.mp4 --> xxx. + srt
    # 如果是其他格式，如mpweg需要改一下，这里因为都是mp4就直接对字符串切片了
    raw_file = file
    save_file = file[:-3] + "srt"
    print (file,raw_file,save_file)
    # 判断文件是否存在，存在则说明已经有字幕，跳出不识别
    if os.path.exists(save_file):
        time.sleep(0.01)
        continue
    # 获取当前视频识别开始时间
    start_time = datetime.datetime.now()
    print('正在识别：{} --{}'.format('\\'.join(file.split('\\')[2:]),start_time.strftime('%Y-%m-%d %H:%M:%S')))
    # 获取视频时长
    video = imageio.get_reader(raw_file)
    duration = seconds_to_hmsm(video.get_meta_data()['duration'])
    video.close()
    print('视频时长：{}'.format(duration))

    # 文字识别
    res = model.transcribe(raw_file,fp16=False,language='English')
    with open(save_file,'w',encoding='utf-8') as f:
        i = 1
        for r in res['segments']:
            f.write(str(i)+'\n')
            f.write(seconds_to_hmsm(float(r['start']))+' --> '+seconds_to_hmsm(float(r['end']))+'\n')
            i += 1
            #font_format = "{\\3c&Hd22c255&}{\\fn黑体\\fs27}"
		    #f.write(font_format+'\n')
            f.write(r['text']+'\n')
		    #f.write(convert(r['text'], 'zh-cn')+'\n') # 结果可能是繁体，转为简体zh-cn
            f.write('\n')
		    # 获取当前视频识别结束时间
    end_time = datetime.datetime.now()
    print('完成识别：{} --{}'.format('\\'.join(file.split('\\')[2:]),end_time.strftime('%Y-%m-%d %H:%M:%S')))
    print('花费时间:',end_time-start_time)

  0%|          | 0/2 [00:00<?, ?it/s]

Sample Management Recording.mp4 Sample Management Recording.mp4 Sample Management Recording.srt
正在识别： --2024-08-16 09:56:47
视频时长：01:33:36,050


 50%|█████     | 1/2 [03:27<03:27, 207.03s/it]

完成识别： --2024-08-16 10:00:14
花费时间: 0:03:27.033731
Data CurationOffice Hours Recording.mp4 Data CurationOffice Hours Recording.mp4 Data CurationOffice Hours Recording.srt
正在识别： --2024-08-16 10:00:14
视频时长：01:02:27,969


100%|██████████| 2/2 [05:34<00:00, 167.25s/it]

完成识别： --2024-08-16 10:02:21
花费时间: 0:02:07.463223
